In [1]:
import pandas as pd
import requests
from sqlalchemy import create_engine


path = '/home/emmanuel/Desktop/descarga_db/raw_data_project_m1.db'
engine = create_engine(f'sqlite:///{path}')
raw_df = pd.read_sql_query("SELECT * FROM sqlite_master WHERE type='table'", engine)

In [2]:
#query to get the df information I want 

df_raw = pd.read_sql("""SELECT personal_info.uuid,
personal_info.gender,
career_info.normalized_job_code,
country_info.country_code
FROM personal_info
JOIN career_info 
ON career_info.uuid = personal_info.uuid
JOIN country_info 
ON country_info.uuid = personal_info.uuid
JOIN poll_info 
ON poll_info.uuid = country_info.uuid 
GROUP BY career_info.uuid;""",engine)
df_raw

,uuid,gender,normalized_job_code,country_code
0,0001afe0-da16-0133-9d54-0a81e8b09a82,male,7932d4705d002b47789c11edfaed685e,NL
1,00073240-d995-0133-f7e1-0a81e8b09a82,male,None,HR
2,0007b2d0-daf3-0133-c5f0-0a81e8b09a82,FeMale,e7b3e1c6a32c7247cd4857e4c3574165,DE
3,00199e00-d8c3-0133-c6b2-0a81e8b09a82,Male,None,IT
4,0028cc20-dcc4-0133-f408-0a81e8b09a82,female,None,FR
...,...,...,...,...
9644,ffe6bd40-d98a-0133-a373-0a81e8b09a82,male,ac47656fd51c2cd8037057262c910dc4,FR
9645,fff34480-db17-0133-381a-0a81e8b09a82,male,ac47656fd51c2cd8037057262c910dc4,DE
9646,fffa2a60-dc71-0133-8480-0a81e8b09a82,female,None,ES
9647,fffcd3a0-d975-0133-b045-0a81e8b09a82,Male,None,LT


In [3]:
def gender_modifier(x):
    if x == 'FeMale':
        return 'Female'
    elif x == 'Fem':
        return 'Female'
    elif x == 'female':
        return 'Female'
    elif x == 'male':
        return 'Male'
    elif x == 'Male':
        return 'Male'



In [4]:
#appliying my definition to clean gender column
df_raw['gender_modified'] = df_raw['gender'].apply(gender_modifier)

#from df raw, dropping the old and unclean 'gender' column
ppl_df_unclean = df_raw.drop(columns=['gender'])

#rename column
ppl_df_clean = ppl_df_unclean.rename(columns={'gender_modified':'gender'})

ppl_df_clean.to_csv('clean_ppl_df.csv',index=False)


In [7]:
#jobs related to normalized_job_code 
#r = requests.get('http://api.dataatwork.org/v1/jobs/ac47656fd51c2cd8037057262c910dc4')
#req = r.json()

#api caller function

def api_caller(x):
    r = requests.get('http://api.dataatwork.org/v1/jobs/' + str(x))
    req = r.json()
    return ["normalized_job_title"]


In [24]:

#df_raw['normalized_job_code'].unique()


In [ ]:
x = df_raw['normalized_job_code'].apply(api_caller)
x


In [94]:
#country codes through webscrapping

url = 'https://www.iban.com/country-codes' 
raw_codes = pd.read_html(url,header=0)[0]
raw_codes

#only keeping columns that are needed
country_codes = raw_codes.drop(columns=['Alpha-3 code','Numeric'])
country_code = country_codes.rename(columns={'Country':'country','Alpha-2 code':'country_code'})

#saving all the country codes
country_code.to_csv('country_codes.csv',index=False)

In [1]:
#merging clean_ppl and country



NameError: name 'pd' is not defined